In [2]:
import os
import sys
import numpy as np
import pandas as pd
    
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
    
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging
from time import time            
    
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
    
mat_file_list = ['arrhythmia.mat',
                    'cardio.mat',
                    'glass.mat',
                    'ionosphere.mat',
                    'letter.mat',
                    'lympho.mat',
                    'mnist.mat',
                    'musk.mat',
                    'optdigits.mat',
                    'pendigits.mat',
                    'pima.mat',
                    'satellite.mat',
                    'satimage-2.mat',
                    'shuttle.mat',
                    'vertebral.mat',
                    'vowels.mat',
                    'wbc.mat']
    
    
df_columns = ['Data','#Samples','#Dimensions','Outlier Perc','ABOD','CBLOF','FB','HBOS','IForest','KNN','LOF','MCD','OCSVM','PCA']
    
roc_df = pd.DataFrame(columns=df_columns)
print(roc_df)
    
time_df = pd.DataFrame(columns=df_columns)
print(time_df)
    
prn_df = pd.DataFrame(columns=df_columns)
print(prn_df)
    
print(os)
random_state = np.random.RandomState(42)
    
for i in mat_file_list:
        mat = loadmat(os.path.join('data', i))
        print("\n...processing", i, '...')
            
            
        X = mat['X']
        y = mat['y'].ravel()
        outliers_fraction = np.count_nonzero(y) / len(y)
        outliers_percentage = round(outliers_fraction * 100, ndigits=4)
            
        roc_list = [i[:-4], X.shape[0], X.shape[1], outliers_percentage]
        prn_list = [i[:-4], X.shape[0], X.shape[1], outliers_percentage]
        time_list = [i[:-4], X.shape[0], X.shape[1], outliers_percentage]
            
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = None)
        X_train_norm, X_test_norm = standardizer(X_train, X_test)
            
        classifiers = {'Angle-based outlier detector (ABOD)': ABOD(contamination = outliers_fraction),
                           'Cluster-based Local outlier factor': CBLOF(contamination= outliers_fraction,check_estimator=False,random_state=random_state, alpha=0.75,beta=3),
                           'feature bagging': FeatureBagging(contamination = outliers_fraction, random_state=None),
                           'HIstogram-base outlier detection (HBOS)': HBOS(contamination=outliers_fraction),
                           'Isolation forest': IForest(contamination=outliers_fraction, random_state=None),
                           'K Nearest neibghours (KNN)': KNN(contamination=outliers_fraction),
                           'Local outlier factor': LOF(contamination=outliers_fraction),
                           'Minimum Covariance determination (MCD)': MCD(contamination=outliers_fraction, random_state=None),
                           'One-class SVM (OCSVM)': OCSVM(contamination = outliers_fraction),
                           'Principal component analysis (PCA)': PCA(contamination = outliers_fraction, random_state=None)}
        for clf_name, clf in classifiers.items():
            t0 = time()
            clf.fit(X_train_norm)
            test_scores = clf.decision_function(X_test_norm)
            t1 = time()
            duration = round(t1 - t0, ndigits=4)
            time_list.append(duration)
            
            roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
            prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
            
            print('{clf_name} ROC:{roc}, precision @ rank n: {prn}, execution time: {duration}s'.format(clf_name=clf_name,
                                                                                                        roc=roc, prn=prn, duration=duration))
            roc_list.append(roc)
            prn_list.append(prn)
        
        temp_df = pd.DataFrame(time_list).transpose()
        temp_df.columns = df_columns
        time_df = pd.concat([time_df, temp_df], axis=0)
    
        temp_df = pd.DataFrame(roc_list).transpose()
        temp_df.columns = df_columns
        roc_df = pd.concat([roc_df, temp_df], axis=0)
    
        temp_df = pd.DataFrame(prn_list).transpose()
        temp_df.columns = df_columns
        prn_df = pd.concat([prn_df, temp_df], axis=0)
    
print("\n the prn_df")
print(prn_df)
print("\n the time_df")
print(time_df)
print("\n the roc_df")
print(roc_df)

Empty DataFrame
Columns: [Data, #Samples, #Dimensions, Outlier Perc, ABOD, CBLOF, FB, HBOS, IForest, KNN, LOF, MCD, OCSVM, PCA]
Index: []
Empty DataFrame
Columns: [Data, #Samples, #Dimensions, Outlier Perc, ABOD, CBLOF, FB, HBOS, IForest, KNN, LOF, MCD, OCSVM, PCA]
Index: []
Empty DataFrame
Columns: [Data, #Samples, #Dimensions, Outlier Perc, ABOD, CBLOF, FB, HBOS, IForest, KNN, LOF, MCD, OCSVM, PCA]
Index: []
<module 'os' from 'C:\\Users\\User\\anaconda3\\lib\\os.py'>

...processing arrhythmia.mat ...
Angle-based outlier detector (ABOD) ROC:0.7098, precision @ rank n: 0.2308, execution time: 1.923s


C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Cluster-based Local outlier factor ROC:0.7311, precision @ rank n: 0.3077, execution time: 1.852s
feature bagging ROC:0.7392, precision @ rank n: 0.2308, execution time: 0.6816s
HIstogram-base outlier detection (HBOS) ROC:0.8011, precision @ rank n: 0.3846, execution time: 2.4298s
Isolation forest ROC:0.748, precision @ rank n: 0.3077, execution time: 0.5367s
K Nearest neibghours (KNN) ROC:0.7248, precision @ rank n: 0.3077, execution time: 0.0979s
Local outlier factor ROC:0.733, precision @ rank n: 0.3077, execution time: 0.0819s


C:\Users\User\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance determination (MCD) ROC:0.6961, precision @ rank n: 0.1538, execution time: 0.9804s
One-class SVM (OCSVM) ROC:0.7548, precision @ rank n: 0.3077, execution time: 0.053s
Principal component analysis (PCA) ROC:0.7486, precision @ rank n: 0.3077, execution time: 0.067s

...processing cardio.mat ...
Angle-based outlier detector (ABOD) ROC:0.5752, precision @ rank n: 0.2321, execution time: 0.7405s
Cluster-based Local outlier factor ROC:0.7141, precision @ rank n: 0.2679, execution time: 0.1489s


...

Angle-based outlier detector (ABOD) ROC:0.9531, precision @ rank n: 0.4286, execution time: 0.6296s
Cluster-based Local outlier factor ROC:0.8261, precision @ rank n: 0.1429, execution time: 0.1799s


C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


feature bagging ROC:0.9377, precision @ rank n: 0.2143, execution time: 0.5577s
HIstogram-base outlier detection (HBOS) ROC:0.6707, precision @ rank n: 0.2143, execution time: 0.006s
Isolation forest ROC:0.7835, precision @ rank n: 0.1429, execution time: 0.5037s
K Nearest neibghours (KNN) ROC:0.9681, precision @ rank n: 0.5, execution time: 0.1019s
Local outlier factor ROC:0.9296, precision @ rank n: 0.2143, execution time: 0.065s
Minimum Covariance determination (MCD) ROC:0.6589, precision @ rank n: 0.0, execution time: 1.5291s
One-class SVM (OCSVM) ROC:0.7984, precision @ rank n: 0.2857, execution time: 0.0859s
Principal component analysis (PCA) ROC:0.6292, precision @ rank n: 0.0714, execution time: 0.005s

...processing wbc.mat ...
Angle-based outlier detector (ABOD) ROC:0.8899, precision @ rank n: 0.0, execution time: 0.2069s
Cluster-based Local outlier factor ROC:0.8991, precision @ rank n: 0.4, execution time: 0.0879s


C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


feature bagging ROC:0.9321, precision @ rank n: 0.4, execution time: 0.1309s
HIstogram-base outlier detection (HBOS) ROC:0.9083, precision @ rank n: 0.4, execution time: 0.013s
Isolation forest ROC:0.9174, precision @ rank n: 0.4, execution time: 0.4387s
K Nearest neibghours (KNN) ROC:0.9156, precision @ rank n: 0.2, execution time: 0.026s
Local outlier factor ROC:0.9321, precision @ rank n: 0.4, execution time: 0.012s
Minimum Covariance determination (MCD) ROC:0.8624, precision @ rank n: 0.4, execution time: 0.1109s
One-class SVM (OCSVM) ROC:0.9339, precision @ rank n: 0.4, execution time: 0.008s
Principal component analysis (PCA) ROC:0.9193, precision @ rank n: 0.4, execution time: 0.003s

 the prn_df
         Data #Samples #Dimensions Outlier Perc    ABOD   CBLOF      FB  \
0  arrhythmia      452         274      14.6018  0.2308  0.3077  0.2308   
0      cardio     1831          21       9.6122  0.2321  0.2679  0.1429   
0       glass      214           9       4.2056       0  0.333

In [ ]:
jupyter_notebook_config